## Imports

In [8]:
import sys
import torch

sys.path.append('../')
from rel_extract.extractor import RelationExtractor
from rel_extract.model import RelTaggerModel
from transformers import BertModel
from transformers import BertTokenizer
import transformers
import pandas as pd
import json

In [7]:
import keras 
keras.__version__

'2.3.1'

In [6]:
import tensorflow as tf
tf.__version__

'1.15.0'

## Loading the model

In [10]:
_filename = 'save_bert_large_with_squad0'
#_filename = 'save_bert_large_without_squad2'

model_class = BertModel
tokenizer_class = BertTokenizer
#pretrained_weights = 'bert-large-uncased'
pretrained_weights = 'bert-large-uncased-whole-word-masking-finetuned-squad'
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, from_tf=True)
language_model = model_class.from_pretrained(_filename)

#model = RelTaggerModel(language_model)
#checkpoint = torch.load(_filename, map_location='cpu')
#model.load_state_dict(checkpoint['model_state_dict'])

KeyboardInterrupt: 

In [12]:
filename = 'save_bert_large_with_squad0'

In [11]:
language_model = model_class.from_pretrained(_filename)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [5]:
model = RelTaggerModel(language_model)
checkpoint = torch.load(_filename)
# map_location='cpu'

In [6]:
model.load_state_dict(checkpoint['model_state_dict'])

RuntimeError: Error(s) in loading state_dict for RelTaggerModel:
	Missing key(s) in state_dict: "language_model.embeddings.position_ids". 

In [ ]:
# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
#tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
#model = model_class.from_pretrained(pretrained_weights)

## Relation extraction examples

In [7]:
tokenizer('John Smith received an OBE')

{'input_ids': [101, 2198, 3044, 2363, 2019, 15578, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [8]:
print(tokenizer.encode('I am a teacher'))                    
sen_code = tokenizer.encode_plus('I am a student','I am not a teacher')
print('--------------')
print(sen_code)

[101, 1045, 2572, 1037, 3836, 102]
--------------
{'input_ids': [101, 1045, 2572, 1037, 3076, 102, 1045, 2572, 2025, 1037, 3836, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [9]:
print(tokenizer.convert_ids_to_tokens(sen_code['input_ids']))

['[CLS]', 'i', 'am', 'a', 'student', '[SEP]', 'i', 'am', 'not', 'a', 'teacher', '[SEP]']


In [10]:
tokens_tensor = torch.tensor([sen_code['input_ids']])          
segments_tensors = torch.tensor([sen_code['token_type_ids']])
model.eval()
#进行编码
#with torch.no_grad():
    #outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    #encoded_layers = outputs   

RelTaggerModel(
  (language_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

### -------------------Examples--------------------------

In [28]:
relations = ['noble title', 'founding date', 'occupation of a person','statement describes','sitter','elector','father','hair color']
#relations =[]

In [29]:
extractor = RelationExtractor(model, tokenizer, relations)

In [30]:
#extractor.rank(text='John Smith is my father', head='John Smith', tail='father')
result=extractor.rank(text='John Smith received an OBE', head='John Smith', tail='OBE')
result

[('noble title', 0.9690611492842436),
 ('elector', 0.021126389503479004),
 ('statement describes', 0.002753138542175293),
 ('father', 0.001619577407836914),
 ('occupation of a person', 0.0012609362602233887),
 ('sitter', 0.00035965442657470703),
 ('hair color', 0.00029224157333374023),
 ('founding date', 0.00024014711380004883)]

In [14]:
result1=pd.DataFrame(result)
result1

,0,1
0,noble title,0.969061
1,elector,0.021126
2,statement describes,0.002753
3,father,0.001620
4,occupation of a person,0.001261
5,sitter,0.000360
6,hair color,0.000292
7,founding date,0.000240


In [24]:
print(result[0])

('noble title', 0.9690611492842436)


### Other Examples

In [32]:
extractor.rank(text='John Smith works as a carpenter', head='John Smith', tail='carpenter')

[('occupation of a person', 0.9911835361272097),
 ('statement describes', 0.05648636817932129),
 ('noble title', 0.0033943653106689453),
 ('elector', 0.0006299614906311035),
 ('father', 0.0006273388862609863),
 ('sitter', 0.00032138824462890625),
 ('hair color', 0.00030422210693359375),
 ('founding date', 0.00026363134384155273)]

In [16]:
extractor.rank(text='Acme Inc. was founded in 1959', head='Acme Inc.', tail='1959')

[('founding date', 0.9776930660009384),
 ('statement describes', 0.05399411916732788),
 ('sitter', 0.011232316493988037),
 ('elector', 0.0018630027770996094),
 ('father', 0.0009074211120605469),
 ('occupation of a person', 0.0006635785102844238),
 ('hair color', 0.0006107091903686523),
 ('noble title', 0.000413358211517334)]

In [17]:
extractor.rank(text='John Smith is my father', head='John Smith', tail='father')

[('father', 0.9124629572033882),
 ('elector', 0.2054157853126526),
 ('statement describes', 0.07709759473800659),
 ('sitter', 0.06288880109786987),
 ('occupation of a person', 0.002827942371368408),
 ('noble title', 0.0018759369850158691),
 ('hair color', 0.00040787458419799805),
 ('founding date', 0.00027865171432495117)]

In [18]:
extractor.rank(text='John Smith has green color hair', head='John Smith', tail='hair')

[('hair color', 0.9843112882226706),
 ('statement describes', 0.365623414516449),
 ('father', 0.009144425392150879),
 ('sitter', 0.001943826675415039),
 ('occupation of a person', 0.0010306835174560547),
 ('elector', 0.000652611255645752),
 ('noble title', 0.00028842687606811523),
 ('founding date', 0.00025767087936401367)]

In [33]:
extractor.rank(text='Donald Trump will run for President', head='Donald Trump', tail='President')

[('elector', 0.9719951674342155),
 ('statement describes', 0.005742073059082031),
 ('sitter', 0.0016294121742248535),
 ('father', 0.0006754398345947266),
 ('occupation of a person', 0.0005661845207214355),
 ('noble title', 0.0003142356872558594),
 ('hair color', 0.00027698278427124023),
 ('founding date', 0.0002523064613342285)]

### ----------------------Label Extraction--------------------------

In [40]:
with open('pid2name.json','r',encoding='utf8')as index:
    json_data = json.load(index)

In [49]:
json_data

{'P2384': ['statement describes',
  'formalization of the statement contains a bound variable in this class'],
 'P2388': ['office held by head of the organization',
  'position of the head of this item'],
 'P2389': ['organization directed from the office or person',
  'No description defined'],
 'P2634': ['sitter',
  'person who posed during the creation of a work, whether or not that person is eventually depicted as oneself'],
 'P607': ['conflict',
  'battles, wars or other military engagements in which the person or item participated'],
 'P608': ['exhibition history',
  'exhibitions where the item is or was displayed'],
 'P609': ['terminus location', 'location of the terminus of a linear feature'],
 'P237': ['coat of arms', "subject's coat of arms"],
 'P2925': ['domain of saint or deity',
  'domain(s) which this saint or deity controls or protects'],
 'P927': ['anatomical location',
  'where in the body does this anatomical feature lie'],
 'P2319': ['elector',
  'people or other enti

In [57]:
json_data1=pd.DataFrame(json_data)
#json_data1
json_data2=json_data1[0:1].values
#json_data2

In [59]:
#index=json_data2.tolist()
index = []
for k in json_data2: 
    for j in k: 
        index.append(j)
index

['statement describes',
 'office held by head of the organization',
 'organization directed from the office or person',
 'sitter',
 'conflict',
 'exhibition history',
 'terminus location',
 'coat of arms',
 'domain of saint or deity',
 'anatomical location',
 'elector',
 'debut participant',
 'direction',
 'presynaptic connection',
 'final event',
 'programming paradigm',
 'board member',
 'signed form',
 'standards body',
 'iconographic symbol',
 'owned by',
 'maintained by',
 'commissioned by',
 'publisher',
 'basic form of government',
 'item operated',
 'connecting line',
 'excavation director',
 'drainage basin',
 'composer',
 'physically interacts with',
 'regulates (molecular biology)',
 'operating area',
 'sidekick of',
 'bowling style',
 'cast member',
 'licensed to broadcast to',
 'language regulatory body',
 'applies to part',
 'powered by',
 'original combination',
 'interleaves with',
 'academic degree',
 'criterion used',
 'honorific prefix',
 'Roman praenomen',
 'metasub

In [61]:
relations_all=index
extractor_all = RelationExtractor(model, tokenizer, relations_all)

In [63]:
extractor_all.rank(text='John Smith works as a carpenter', head='John Smith', tail='carpenter')

[('members have occupation', 0.9889186322689056),
 ('field of this occupation', 0.986709157936275),
 ('field of work', 0.9860145403072238),
 ('occupation', 0.9855738850310445),
 ('for work', 0.9760994222015142),
 ('domain', 0.9716210756450891),
 ('industry', 0.9641974791884422),
 ('fabrication method', 0.9638764038681984),
 ('subject has role', 0.9517217352986336),
 ("topic's main category", 0.9471006579697132),
 ('lifestyle', 0.9367808997631073),
 ('subclass of', 0.9308632016181946),
 ('category combines topics', 0.9244979172945023),
 ("category's main topic", 0.920000858604908),
 ('exemplar of', 0.911686047911644),
 ('has parts of the class', 0.8907510191202164),
 ('facet of', 0.8835225328803062),
 ('molecular function', 0.8832404986023903),
 ('instance of', 0.8684815317392349),
 ('armament', 0.8656913787126541),
 ('health specialty', 0.8515726178884506),
 ('academic degree', 0.8495136797428131),
 ('studies', 0.7871876657009125),
 ('anatomical branch of', 0.6941528618335724),
 ('mono